In [11]:
import mujoco
import mujoco.viewer
import numpy as np
import time
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# Load the Go2 quadruped model
# ------------------------------------------------------------
model = mujoco.MjModel.from_xml_path("/Users/gagandeshad/Desktop/unitree/mujoco_menagerie/unitree_go2/scene.xml")
data = mujoco.MjData(model)

# ------------------------------------------------------------
# Gait parameters
# ------------------------------------------------------------
A_hip = 0.2        # amplitude for hip (radians)
A_thigh = 0.8      # amplitude for thigh
A_knee_offset = -1.8
A_knee_swing = 0.7
freq = 2.0         # stepping frequency (Hz)
omega = 2 * np.pi * freq

# Phase offsets (trot gait)
# FL & RR in phase; FR & RL in opposite phase
phase = {
    "FL": 0,
    "RR": 0,
    "FR": np.pi,
    "RL": np.pi
}

# ------------------------------------------------------------
# Logging setup
# ------------------------------------------------------------
pos_log = []
vel_log = []
time_log = []

# ------------------------------------------------------------
# Simulation setup
# ------------------------------------------------------------
sim_duration = 10.0  # seconds
timestep = model.opt.timestep
n_steps = int(sim_duration / timestep)

print(f"Simulating for {sim_duration:.1f} seconds ({n_steps} steps)...")

# ------------------------------------------------------------
# Launch viewer
# ------------------------------------------------------------
with mujoco.viewer.launch_passive(model, data) as viewer:
    start_time = time.time()

    for i in range(n_steps):
        t = i * timestep

        # Apply trot control to each leg
        for leg in ["FL", "FR", "RL", "RR"]:
            phi = phase[leg]

            hip_cmd = 0.05 * np.sin(omega * t + phi)            # small abduction
            thigh_cmd = -A_thigh * np.sin(omega * t + phi)      # forward swing (note negative)
            knee_cmd = A_knee_offset + A_knee_swing * np.sin(omega * t + phi + np.pi)

            data.ctrl[model.actuator(f"{leg}_hip").id] = hip_cmd
            data.ctrl[model.actuator(f"{leg}_thigh").id] = thigh_cmd
            data.ctrl[model.actuator(f"{leg}_calf").id] = knee_cmd

        # Step simulation
        mujoco.mj_step(model, data)

        # Log base position and velocity along x-axis
        pos_x = data.qpos[0]       # base x position
        vel_x = data.qvel[0]       # base x velocity
        pos_log.append(pos_x)
        vel_log.append(vel_x)
        time_log.append(t)

        # Update viewer
        viewer.sync()
        if viewer.is_running() is False:
            break

        # Optional real-time pacing
        elapsed = time.time() - start_time
        if t > elapsed:
            time.sleep(t - elapsed)

# ------------------------------------------------------------
# Plot logged data
# ------------------------------------------------------------
plt.figure(figsize=(10, 5))
plt.subplot(2, 1, 1)
plt.plot(time_log, pos_log, label="Position (x)")
plt.ylabel("Position (m)")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(time_log, vel_log, label="Velocity (x)", color='orange')
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.legend()
plt.grid(True)

plt.suptitle("Go2 Forward Trot Gait – Position and Velocity Over Time")
plt.tight_layout()
plt.show()


Simulating for 10.0 seconds (5000 steps)...


RuntimeError: `launch_passive` requires that the Python script be run under `mjpython` on macOS

In [1]:
import mujoco
import mujoco.viewer
import numpy as np
import os

In [2]:
base_path = "/Users/gagandeshad/Desktop/unitree/mujoco_menagerie"
model_path = f"{base_path}/unitree_go2/scene.xml"

In [3]:
print(f"Loading model: {model_path}")
model = mujoco.MjModel.from_xml_path(model_path)
data = mujoco.MjData(model)

Loading model: /Users/gagandeshad/Desktop/unitree/mujoco_menagerie/unitree_go2/scene.xml


In [4]:
# identify roles by name so we use the hip-pitch DOF (fore/aft) not hip-yaw (turning)
def identify_leg_roles(model, leg_joints):
    roles = {}
    for leg, jids in leg_joints.items():
        roles[leg] = {'hip_pitch': None, 'thigh': None, 'calf': None}
        for jid in jids:
            nm = model.joint(jid).name.lower()
            if 'hip' in nm and 'pitch' in nm:
                roles[leg]['hip_pitch'] = jid
            elif 'thigh' in nm or 'femur' in nm or 'upper' in nm:
                roles[leg]['thigh'] = jid
            elif 'calf' in nm or 'knee' in nm:
                roles[leg]['calf'] = jid
        # fallback: if hip_pitch not found, use first 'hip' containing joint or first joint
        if roles[leg]['hip_pitch'] is None:
            for jid in jids:
                if 'hip' in model.joint(jid).name.lower():
                    roles[leg]['hip_pitch'] = jid
                    break
        if roles[leg]['thigh'] is None and len(jids) >= 2:
            roles[leg]['thigh'] = jids[1]
        if roles[leg]['calf'] is None and len(jids) >= 3:
            roles[leg]['calf'] = jids[2]
    return roles

roles = identify_leg_roles(model, leg_joints)
print("Detected leg roles:", {k: {r: (model.joint(v).name if v is not None else None)
                                for r, v in roles[k].items()} for k in roles})


NameError: name 'leg_joints' is not defined

In [5]:
print("\nAvailable joints:")
leg_joints = {}
for i in range(model.njnt):
    jname = model.joint(i).name
    print(f"  [{i}] {jname}")


Available joints:
  [0] 
  [1] FL_hip_joint
  [2] FL_thigh_joint
  [3] FL_calf_joint
  [4] FR_hip_joint
  [5] FR_thigh_joint
  [6] FR_calf_joint
  [7] RL_hip_joint
  [8] RL_thigh_joint
  [9] RL_calf_joint
  [10] RR_hip_joint
  [11] RR_thigh_joint
  [12] RR_calf_joint


In [6]:
if jname.startswith('FL_'):
    leg_joints.setdefault('FL', []).append(i)
elif jname.startswith('FR_'):
    leg_joints.setdefault('FR', []).append(i)
elif jname.startswith('RL_'):
    leg_joints.setdefault('RL', []).append(i)
elif jname.startswith('RR_'):
    leg_joints.setdefault('RR', []).append(i)

In [7]:
for leg in leg_joints:
    leg_joints[leg] = sorted(leg_joints[leg], key=lambda j: model.jnt_qposadr[j])
    
print(f"\nLeg joints grouped: {leg_joints}")


Leg joints grouped: {'RR': [12]}


In [9]:
# identify roles by name so we use the hip-pitch DOF (fore/aft) not hip-yaw (turning)
def identify_leg_roles(model, leg_joints):
    roles = {}
    for leg, jids in leg_joints.items():
        roles[leg] = {'hip_pitch': None, 'thigh': None, 'calf': None}
        for jid in jids:
            nm = model.joint(jid).name.lower()
            if 'hip' in nm and 'pitch' in nm:
                roles[leg]['hip_pitch'] = jid
            elif 'thigh' in nm or 'femur' in nm or 'upper' in nm:
                roles[leg]['thigh'] = jid
            elif 'calf' in nm or 'knee' in nm:
                roles[leg]['calf'] = jid
        # fallback: if hip_pitch not found, use first 'hip' containing joint or first joint
        if roles[leg]['hip_pitch'] is None:
            for jid in jids:
                if 'hip' in model.joint(jid).name.lower():
                    roles[leg]['hip_pitch'] = jid
                    break
        if roles[leg]['thigh'] is None and len(jids) >= 2:
            roles[leg]['thigh'] = jids[1]
        if roles[leg]['calf'] is None and len(jids) >= 3:
            roles[leg]['calf'] = jids[2]
    return roles

roles = identify_leg_roles(model, leg_joints)
print("Detected leg roles:", {k: {r: (model.joint(v).name if v is not None else None)
                                for r, v in roles[k].items()} for k in roles})


Detected leg roles: {'RR': {'hip_pitch': None, 'thigh': None, 'calf': 'RR_calf_joint'}}
